In [1]:
import re
import time

import numpy as np
import pandas as pd
import requests

In [2]:
!pip install progressbar2
from progressbar import progressbar

  Using cached https://files.pythonhosted.org/packages/16/68/adc395e0a3c86571081c8a2e2daaa5b58270f6854276a089a0e9b5fa2c33/progressbar2-3.47.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/eb/a0/19119d8b7c05be49baf6c593f11c432d571b70d805f2fe94c0585e55e4c8/python_utils-2.3.0-py2.py3-none-any.whl


## Scraping

### Scraping Functions

In [7]:
# function to scrape reddit page (takes a reddit .json url)
# returns posts 

def scraper_bike(url):
    headers = {'User-Agent' : 'override this bad boy!'}
    posts = []
    after = {}

    for page in progressbar(range(40)):
        params = {'after': after}
        pagepull = requests.get(url=url, params=params, headers=headers)
        page_dict = pagepull.json()
        posts.extend(page_dict['data']['children'])
        after = page_dict['data']['after']
        time.sleep(.2)
        
    return posts

In [8]:
# function to convert posts to DataFrame - won't allow duplicate posts since unique id 'name' is set as index
# Extract: name (as index) and subreddit, selftext, title (as columns)

def posts_to_df(post_list):
    i = 0
    post_dict = {}
    
    for post in post_list:
        ind = post_list[i]['data']
        post_dict[ind['name']] = [ind['subreddit'], ind['title'], ind['selftext']]
        i += 1

    df_name = pd.DataFrame(post_dict)
    df_name = df_name.T
    df_name.columns = ['subreddit', 'title', 'selftext']
    
    return df_name

In [9]:
# takes scraper function and url - outputs dataframe

def scrape_to_df(scrape_func, url):
    
    return posts_to_df(scrape_func(url))

In [10]:
#### If you want to scrape repeatedly over time and add to a csv
# scrape, import csv, concat, drop duplicate, and output to csv
# takes in scraper function, url, csv filename to import, csv filename to output
# Outputs - Concatenated DataFrame as csv

def scrape_add(scrape_func, url, import_file, export_file):
    scrape_df = posts_to_df(scrape_func(url))
    imported_df = pd.read_csv(import_file, index_col = 'Unnamed: 0')
    concat_df = pd.concat([imported_df, scrape_df])
    concat_df = concat_df[~concat_df.index.duplicated(keep='first')]
    concat_df.to_csv(export_file)

### Run Scrape

In [11]:
# You can also put in any 2 subreddits in as the URL and get results for those

nfltest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nfl.json')
nbatest = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/nba.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:20 Time:  0:00:20
100% (40 of 40) |########################| Elapsed Time: 0:00:22 Time:  0:00:22


In [12]:
politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

100% (40 of 40) |########################| Elapsed Time: 0:00:25 Time:  0:00:25
100% (40 of 40) |########################| Elapsed Time: 0:00:22 Time:  0:00:22


In [13]:
nbatest.shape

(785, 3)

In [14]:
nfltest.shape

(913, 3)

##### These scrape_add functions add to already built csvs

In [ ]:
# scrape_add(scraper_bike, 'https://www.reddit.com/r/CollegeBasketball/new.json', 'NCAA_Posts_Update2.csv', 'NCAA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/AskScience/new.json', 'AskSci_Posts_Update2.csv', 'AskSci_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nba/new.json', 'NBA_Posts_Update2.csv', 'NBA_Posts_Update3.csv')
# scrape_add(scraper_bike, 'https://www.reddit.com/r/nfl/new.json', 'NFL_Posts_Update2.csv', 'NFL_Posts_Update3.csv')

### Data Cleaning / Preprocessing

In [15]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

pd.set_option('max_colwidth', 300)

In [16]:
nfltest

,subreddit,title,selftext
t3_f2b7iv,nfl,Talko Tuesday,"Welcome to today's open thread, where /r/nfl users can discuss anything they wish not related directly to the NFL.\n\nWant to talk about personal life? Cool things about your fandom? Whatever happens to be dominating today's news cycle? Do you have something to talk about that didn't warrant its..."
t3_f1ucwm,nfl,r/NFL Top 100 of the 2019 Season - CALL FOR RANKERS,"Greetings /r/NFL! \n\nThe preeminent Top 100 is starting again this year and I am returning as host for the second consecutive year! \n \nIn order to continue with this now **5 week** list (**two** weekly reveal posts) and with the things we will do with it, we have to start ***NOW***. So wit..."
t3_f2sslg,nfl,[Withers] AP source says Browns star DE Myles Garrett to be reinstated by NFL from suspension today.,
t3_f2i0bo,nfl,[Emmanuel Sanders] 😂 Its crazy how people keep talking bout my age but I’m still flying past 23 year olds and only had 2 drops the entire year. 1 with the broncos and one with the niners. Keep bringing up my age to make yourself feel good but go look at the film.,
t3_f2r86r,nfl,"Vikings OC Gary Kubiak excited to be back calling plays: “I never lost the joy. I just said, 'Coach, do you mind if I go back home and mow some grass for a couple days or something to think about it?’ I think I called him in about a day and a half. I was just really excited to do it.""",
...,...,...,...
t3_ey0hik,nfl,[Fortuna] The last three teams to enter the fourth quarter of the Super Bowl with a double-digit deficit have all won.,
t3_eyel2c,nfl,"What, in your opinion, is the biggest snub for Superbowl MVP?",
t3_ey1nof,nfl,"Mahomes becomes the first QB in NFL history to win the Super Bowl, SB MVP, NFL MVP, and FIRST TEAM ALL PRO in his first two seasons as a starter",And no other QB since Marino has even come close to this
t3_exy5di,nfl,"[Highlight] Jimmy G throws up a duck, Breeland picks it off",


In [17]:
# drop column

nfltest = nfltest.drop(columns = 'selftext')
nbatest = nbatest.drop(columns = 'selftext')

In [18]:
# merge subreddit data

train = pd.concat([nfltest, nbatest])

In [20]:
train[-100:]

,subreddit,title
t3_f1bkps,nba,"[Highlight] 17 years ago today, Jordan and Kobe battled and trash-talked during Mike’s final All-Star Game with no fear from either side"
t3_f20izj,nba,What’s the best duo from this century where both players won’t make the Hall of Fame?
t3_f21sb2,nba,Nick young or J.R. ?
t3_f23bsh,nba,How do technical foul free throws factor into TS%
t3_f2gcrr,nba,Embiid Discussion - Can someone knowledgable please help me out with some clarification around the possibility of a trade post deadline?
...,...,...
t3_f1s91b,nba,What seed/how far would this team go in the West if they were coached by Brad Stevens
t3_f1m0cb,nba,People need to understand the NBA and team sports in general are about matchups.
t3_f1nv9u,nba,Do you think the Raptors have a shot of taking out the Bucks in the playoffs?
t3_f20oyn,nba,"Other than Lakers, Clippers, and Bucks, who else has a solid chance at the finals?"


##### Tokenize (grab only word characters)

In [21]:
word_tokenizer = RegexpTokenizer(r'\w+')

In [22]:
train['title'] = train['title'].map(lambda x: word_tokenizer.tokenize(x.lower()))

In [23]:
# rejoin list of tokenized words into single string for each row

train['title'] = train['title'].map(lambda x: ' '.join(x))

In [26]:
train.head()

,subreddit,title
t3_f2b7iv,nfl,talko tuesday
t3_f1ucwm,nfl,r nfl top 100 of the 2019 season call for rankers
t3_f2sslg,nfl,withers ap source says browns star de myles garrett to be reinstated by nfl from suspension today
t3_f2i0bo,nfl,emmanuel sanders its crazy how people keep talking bout my age but i m still flying past 23 year olds and only had 2 drops the entire year 1 with the broncos and one with the niners keep bringing up my age to make yourself feel good but go look at the film
t3_f2r86r,nfl,vikings oc gary kubiak excited to be back calling plays i never lost the joy i just said coach do you mind if i go back home and mow some grass for a couple days or something to think about it i think i called him in about a day and a half i was just really excited to do it


In [ ]:
train['title'][0:5]

### Train test split and converting series to list of strings then to array

In [27]:
X = train[['title']]
y = train['subreddit']

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42,
                                                    stratify=y)

In [29]:
# baseline is

y.value_counts(normalize=True)

nfl    0.537691
nba    0.462309
Name: subreddit, dtype: float64

In [30]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)

In [31]:
len(clean_train_data)

1273

In [32]:
clean_train_data[:5]

['xfl ticket prices in washington are higher than comparable redskins tickets',
 'zach lavine 41 points 8 threes full highlights bulls vs wizards february 11 2020',
 'despite having been selected to first team all pro once 2018 and second team all pro three times 2019 2017 2016 chiefs rt mitchell schwartz has never gone to a pro bowl',
 'if the nfl goes to 18 games where do the extra 2 come from',
 'kansas city celebration detected by national weather service radar']

In [33]:
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [ ]:
len(clean_test_data)

### Count Vectorizer

In [34]:
# instantiate our CountVectorizer. This counts the number of appearances of all the words in our training data and
# eliminates common english stop words. 5000 max features works well for our purposes (tested various numbers). Our
# data is already preprocessed and tokenized manually earlier. ngram_range is 1,3, although all or nearly all our
# features are single words

vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

In [35]:
# fit our training data and test data lists to our count_vectorizer

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

In [36]:
# convert to array

train_data_features = train_data_features.toarray()

In [37]:
# check shapes

train_data_features.shape, test_data_features.shape

((1273, 5000), (425, 5000))

In [38]:
# I wanted check that the features corpus was as expected - removed print statement for readability

vocab = vectorizer.get_feature_names()

In [39]:
vocab[0:200]

['000',
 '02',
 '02 06',
 '06',
 '07',
 '10',
 '10 2020',
 '10 fgm',
 '10 game',
 '10 game span',
 '10 games',
 '10 nfl',
 '10 points',
 '10 seasons',
 '100',
 '100 nfl',
 '100 pass',
 '100 pass blocking',
 '101',
 '101 players',
 '101 players 2019',
 '103',
 '105',
 '106',
 '10th',
 '10th straight',
 '11',
 '11 2020',
 '11 points',
 '111',
 '113',
 '114',
 '116',
 '116 105',
 '11th',
 '12',
 '120',
 '123',
 '126',
 '13',
 '13 kansas',
 '13 kansas city',
 '133',
 '133 92',
 '135',
 '14',
 '140',
 '142',
 '15',
 '15 39',
 '15 game',
 '15 game winning',
 '15 games',
 '150',
 '150 teams',
 '150 teams 2020',
 '15th',
 '16',
 '17',
 '17 18',
 '17th',
 '18',
 '18 33',
 '18 million',
 '18 points',
 '19',
 '19 35',
 '1981',
 '1986',
 '1996',
 '19th',
 '1st',
 '1st round',
 '1st round pick',
 '1st team',
 '20',
 '20 point',
 '20 points',
 '20 years',
 '200',
 '2000',
 '2004',
 '2006',
 '2009',
 '2010',
 '2010s',
 '2011',
 '2012',
 '2013',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2019 nfl'

## MODELING

### Logistic Regression

In [40]:
from sklearn.linear_model import LogisticRegression

In [62]:
# fit logistic regression model

lr = LogisticRegression()

In [63]:
# shape check

train_data_features.shape, y_train.shape

((1273, 5000), (1273,))

In [64]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
lr.score(train_data_features, y_train)

0.997643362136685

In [66]:
lr.score(test_data_features, y_test)

0.9152941176470588

### Feature comparison

Creates a dataframe that matches features to coefficients

In [67]:
coef_list = lr.coef_.tolist()

In [68]:
coef_list = coef_list[0]

In [69]:
coef_df = pd.DataFrame({'features': vectorizer.get_feature_names(),
                        'coefs': coef_list})

In [70]:
coef_df.sort_values(by = ['coefs'])

,features,coefs
1942,nba,-2.231317
911,embiid,-1.289872
1471,knicks,-1.216775
1222,highlight,-1.213947
4911,westbrook,-1.105111
...,...,...
1017,football,1.216912
4600,super,1.242598
611,chiefs,1.283339
3218,qb,1.539649


### Let's throw out these unfair words and rerun

In [71]:
stopwords = set(stopwords.words('english'))

extra_stopwords = ['nba', 'basketball', 'football', 'nfl']

stopwords.update(extra_stopwords)

In [72]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = stopwords,
                             max_features = 5000,
                             ngram_range = (1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1273, 5000), (425, 5000))

In [73]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
lr.score(train_data_features, y_train)

0.9913589945011784

In [76]:
lr.score(test_data_features, y_test)

0.8941176470588236

In [110]:
coef_list = lr.coef_.tolist()
coef_list = coef_list[0]

coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])
coef_df.loc[coef_df.features == 'td']

,features,coefs
4216,td,0.235329


### Decision Tree

In [83]:
from sklearn.tree import DecisionTreeClassifier

In [88]:
tree = DecisionTreeClassifier(max_features=100)

In [89]:
tree.fit(train_data_features, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=100, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [90]:
tree.score(train_data_features, y_train)

0.997643362136685

In [91]:
tree.score(test_data_features, y_test)

0.7223529411764706

### Random Forest

In [92]:
from sklearn.ensemble import RandomForestClassifier

In [93]:
forest = RandomForestClassifier(n_estimators = 100)

In [94]:
forest.fit(train_data_features, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [95]:
forest.score(train_data_features, y_train)

0.997643362136685

In [96]:
forest.score(test_data_features, y_test)

0.8870588235294118

###  Matrix on Logistic Regression

In [97]:
from sklearn.metrics import confusion_matrix

In [98]:
y_pred = lr.predict(test_data_features)

In [99]:
cm = confusion_matrix(y_test, y_pred)

In [100]:
cm_df = pd.DataFrame(cm,
                    columns=['predict_neg', 'predict_pos'],
                    index = ['actual_neg', 'actual_pos'])

In [101]:
cm_df

,predict_neg,predict_pos
actual_neg,176,20
actual_pos,25,204


## Checking where our model failed

In [102]:
comparison_df = pd.DataFrame({'y_actual' : y_test,
             'y_predicted' : y_pred})

In [103]:
mismatch_df = comparison_df[comparison_df['y_actual'] != comparison_df['y_predicted']]

In [104]:
mismatch2_df = pd.concat([mismatch_df, X_test], axis = 1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [105]:
# All incorrect predictions with titles

mismatches = mismatch2_df.dropna()

In [106]:
mismatches

,y_actual,y_predicted,title
t3_ey09cu,nfl,nba,highlight williams runs it in for the td
t3_ey0p03,nfl,nba,highlight how bout those chieeeeeeeeeeefs
t3_eyazqy,nfl,nba,kosko mitchell schwartz had a postseason grade of 92 8 the best grade of any player at any position he allowed just one pressure across three games
t3_eyh22q,nfl,nba,highlight the 49er faithful cheer on the team as they arrive in the bay area
t3_eyt34u,nfl,nba,talko tuesday
t3_ez0d39,nfl,nba,what if the nfl hockey fied penalties
t3_ez1ba5,nfl,nba,weirdest nfl rivalries
t3_ezem3q,nfl,nba,duval coalition formed opposition to removing jacksonville home games
t3_ezgbbt,nfl,nba,what is sutton trade value
t3_ezhbqh,nfl,nba,players tribune what the hell happened to vince young by vince young


### Let's try TF-IDF

Term Frequency / Inverse Document Frequency

TF(w) = (Number of times term w appears in a document) / (Total number of terms in the document)

IDF(w) = log_e(Total number of documents / Number of documents with term w in it)

In [111]:
tfidf_vec = TfidfVectorizer(analyzer="word",
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=stopwords,
                            max_features=5000,
                            ngram_range=(1, 3))

In [112]:
train_data_features = tfidf_vec.fit_transform(clean_train_data)

test_data_features = tfidf_vec.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

((1273, 5000), (425, 5000))

In [113]:
lr.fit(train_data_features, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [114]:
lr.score(train_data_features, y_train)

0.9795758051846033

In [115]:
lr.score(test_data_features, y_test)

0.9011764705882352

### Let's try on some other subreddits

In [116]:
train = pd.concat([politics_test, conservative_test])

In [117]:
X = train[['title']]
y = train['subreddit']

In [118]:
# politics_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/politics.json')
# conservative_test = scrape_to_df(scraper_bike, 'https://www.reddit.com/r/conservative.json')

In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)

In [120]:
politics_test = politics_test.drop(columns='selftext')
conservative_test = conservative_test.drop(columns='selftext')

train = pd.concat([politics_test, conservative_test])
tokenizer = RegexpTokenizer(r'\w+')

train['title'] = train['title'].map(lambda x: tokenizer.tokenize(x.lower()))
train['title'] = train['title'].map(lambda x: ' '.join(x))

In [121]:
# create our training data list - this is a list of strings, with each string being a post title

clean_train_data = []

for traindata in X_train['title']:
    clean_train_data.append(traindata)
    
    
# create test data list

clean_test_data = []

for testdata in X_test['title']:
    clean_test_data.append(testdata)

In [122]:
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words='english',
                             max_features=5000,
                             ngram_range=(1, 3))

train_data_features = vectorizer.fit_transform(clean_train_data)

test_data_features = vectorizer.transform(clean_test_data)

train_data_features = train_data_features.toarray()

train_data_features.shape, test_data_features.shape

vocab = vectorizer.get_feature_names()

### Modeling

In [123]:
lr = LogisticRegression(penalty = 'l2')

In [124]:
train_data_features.shape, y_train.shape

((1297, 5000), (1297,))

In [125]:
lr.fit(train_data_features, y_train)

lr.score(train_data_features, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9830377794911334

In [126]:
lr.score(test_data_features, y_test)

0.7344110854503464

In [127]:
coef_list = lr.coef_.tolist()

coef_list = coef_list[0]

In [128]:
coef_df = pd.DataFrame({'features' : vectorizer.get_feature_names(),
                       'coefs' : coef_list})

coef_df.sort_values(by = ['coefs'])

,features,coefs
692,dems,-1.076319
2678,people,-1.053654
4956,won,-1.018671
4841,video,-1.007651
4462,socialism,-0.949057
...,...,...
1658,military,1.047845
318,bloomberg,1.148311
4527,stone,1.214183
4698,trump,1.280603


# Understanding word vectors

... for, like, actual poets. By [Allison Parrish](http://www.decontextualize.com/)


## Why word vectors?

Poetry is, at its core, the art of identifying and manipulating linguistic similarity. I have discovered a truly marvelous proof of this, which this notebook is too narrow to contain. (By which I mean: I will elaborate on this some other time)

## Animal similarity and simple linear algebra

We'll begin by considering a small subset of English: words for animals. Our task is to be able to write computer programs to find similarities among these words and the creatures they designate. To do this, we might start by making a spreadsheet of some animals and their characteristics. For example:

![Animal spreadsheet](http://static.decontextualize.com/snaps/animal-spreadsheet.png)

This spreadsheet associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. (The values themselves are simply based on my own judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them.)

These values give us everything we need to make determinations about which animals are similar (at least, similar in the properties that we've included in the data). Try to answer the following question: Which animal is most similar to a capybara? You could go through the values one by one and do the math to make that evaluation, but visualizing the data as points in 2-dimensional space makes finding the answer very intuitive:

![Animal space](http://static.decontextualize.com/snaps/animal-space.png)

The plot shows us that the closest animal to the capybara is the panda bear (again, in terms of its subjective size and cuteness). One way of calculating how "far apart" two points are is to find their *Euclidean distance*. (This is simply the length of the line that connects the two points.) For points in two dimensions, Euclidean distance can be calculated with the following Python function:

In [129]:
import numpy as np
def distance2d(x1, y1, x2, y2):
    return np.linalg.norm(np.array([x1, y1])-np.array([x2, y2]))

So, the distance between "capybara" (70, 30) and "panda" (74, 40):

In [130]:
distance2d(70, 30, 75, 40) # panda and capybara

11.180339887498949

... is less than the distance between "tarantula" and "elephant":

In [131]:
distance2d(8, 3, 65, 90) # tarantula and elephant

104.0096149401583

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.

Reasoning visually, you can also answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.)

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

![Animal analogy](http://static.decontextualize.com/snaps/animal-space-analogy.png)

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    
A sequence of numbers used to identify a point is called a *vector*, and the kind of math we've been doing so far is called *linear algebra.* (Linear algebra is surprisingly useful across many domains: It's the same kind of math you might do to, e.g., simulate the velocity and acceleration of a sprite in a video game.)

A set of vectors that are all part of the same data set is often called a *vector space*. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Download the color data and put it in the same directory as this notebook.

A few notes before we proceed:

* The linear algebra functions implemented below (`addv`, `meanv`, etc.) are slow, potentially inaccurate, and shouldn't be used for "real" code—I wrote them so beginner programmers can understand how these kinds of functions work behind the scenes. Use [numpy](http://www.numpy.org/) for fast and accurate math in Python.
* If you're interested in perceptually accurate color math in Python, consider using the [colormath library](http://python-colormath.readthedocs.io/en/latest/).

Now, import the `json` library and load the color data:

In [132]:
import requests

In [133]:
resp = requests.get('https://raw.githubusercontent.com/dariusk/corpora/master/data/colors/xkcd.json')
color_data = resp.json()

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [134]:
def hex_to_int(s):
    return int(s[1:3], 16), int(s[3:5], 16), int(s[5:7], 16)

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [135]:
colors = dict()
for item in color_data['colors']:
    colors[item["color"]] = hex_to_int(item["hex"])

Testing it out:

In [136]:
colors['olive']

(110, 117, 14)

In [137]:
colors['red']

(229, 0, 0)

In [138]:
colors['black']

(0, 0, 0)

In [ ]:
colors.keys()

### Vector math

Before we keep going, we'll need some functions for performing basic vector arithmetic. These functions will work with vectors in spaces of any number of dimensions.

The first function returns the Euclidean distance between two points:

In [139]:
import math
def distance(coord1, coord2):
    return np.linalg.norm(np.array(coord1)-np.array(coord2))
distance([10, 1], [5, 2])

5.0990195135927845

The `subtractv` function subtracts one vector from another:

In [140]:
def subtractv(coord1, coord2):
    return list(np.array(coord1) - np.array(coord2))
subtractv([10, 1], [5, 2])

[5, -1]

The `addv` vector adds two vectors together:

In [141]:
def addv(coord1, coord2):
    return list(np.array(coord1) + np.array(coord2))
addv([10, 1], [5, 2])

[15, 3]

And the `meanv` function takes a list of vectors and finds their mean or average:

In [142]:
def meanv(coords):
    return list(np.mean(coords, axis=0))
meanv([[0, 1], [2, 2], [4, 3]])

[2.0, 2.0]

Just as a test, the following cell shows that the distance from "red" to "green" is greater than the distance from "red" to "pink":

In [143]:
distance(colors['red'], colors['green']) > distance(colors['red'], colors['pink'])

True

### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to farthest. The `closest()` function below does just that. By default, it returns a list of the ten closest items to the given vector.

> Note: Calculating "closest neighbors" like this is fine for the examples in this notebook, but unmanageably slow for vector spaces of any appreciable size. As your vector space grows, you'll want to move to a faster solution, like SciPy's [kdtree](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.KDTree.html) or [Annoy](https://pypi.python.org/pypi/annoy).

In [144]:
def closest(space, coord, n=10):
    return sorted(space.keys(), key=lambda x: distance(coord, space[x]))[:n]

Testing it out, we can find the ten colors closest to "red":

In [145]:
closest(colors, colors['white'])

['white',
 'pale grey',
 'very light pink',
 'off white',
 'ice blue',
 'very pale blue',
 'ice',
 'very light blue',
 'really light blue',
 'eggshell']

... or the ten colors closest to (150, 60, 150):

In [148]:
closest(colors, [5000, 60, 150])

['bubble gum pink',
 'strong pink',
 'electric pink',
 'bubblegum',
 'hot pink',
 'coral pink',
 'carnation pink',
 'salmon',
 'light red',
 'pink']

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [150]:
closest(colors, subtractv(colors['purple'], colors['blue']))

['dark red',
 'blood',
 'deep red',
 'blood red',
 'indian red',
 'reddy brown',
 'mahogany',
 'dried blood',
 'crimson',
 'brick red']

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [151]:
closest(colors, addv(colors['blue'], colors['green']))

['bright turquoise',
 'bright light blue',
 'bright aqua',
 'cyan',
 'neon blue',
 'aqua blue',
 'bright cyan',
 'bright sky blue',
 'aqua',
 'bright teal']

That's right, it's something like turquoise or cyan! What if we find the average of black and white? Predictably, we get gray:

In [152]:
# the average of black and white: medium grey
closest(colors, meanv([colors['black'], colors['white']]))

['medium grey',
 'purple grey',
 'steel grey',
 'battleship grey',
 'grey purple',
 'purplish grey',
 'greyish purple',
 'steel',
 'warm grey',
 'green grey']

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [155]:
# an analogy: pink is to red as X is to blue
pink_to_red = subtractv(colors['pink'], colors['red'])
closest(colors, subtractv(pink_to_red, colors['blue']))

['forrest green',
 'dark green',
 'racing green',
 'hunter green',
 'very dark green',
 'bottle green',
 'darkgreen',
 'dark forest green',
 'dark olive',
 'forest green']

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [156]:
# another example: 
navy_to_blue = subtractv(colors['navy'], colors['blue'])
closest(colors, addv(navy_to_blue, colors['green']))

['true green',
 'dark grass green',
 'grassy green',
 'racing green',
 'forest',
 'bottle green',
 'dark olive green',
 'darkgreen',
 'forrest green',
 'grass green']

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Interlude: A Love Poem That Loses Its Way

In [ ]:
import random
red = colors['red']
blue = colors['blue']
for i in range(14):
    rednames = closest(colors, red)
    bluenames = closest(colors, blue)
    print(f"Roses are {rednames[0]}, violets are {bluenames[0]}")
    red = colors[random.choice(rednames[1:])]
    blue = colors[random.choice(bluenames[1:])]

### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Bram Stoker's *Dracula*.

First, we'll load [spaCy](https://spacy.io/):

In [158]:
!pip install spacy

     |████████████████████████████████| 14.2MB 3.4MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/b9/bd/faace403086ee922afc74e5615cb8c21020fcf5d5667314e943c08f71fde/murmurhash-1.0.2-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/c7/3e/9aaba1f8c0cb69e57ebeb411cc1b65b3f6bfc3572dd68969a6d3e59288f6/preshed-3.0.2-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
     |████████████████████████████████| 276kB 18.3MB/s eta 0:00:01
     |████████████████████████████████| 61kB 25.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/85/d8/f0be9d8ebec9cbeea1427de6ac0ecc919c0bfe881eff2d2965dbc310ca8b/blis-0.4.1-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
     |████████████████████████████████| 3.0MB 1.5MB/s eta 0:00:01

In [161]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0MB 2.3MB/s eta 0:00:01


In [201]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

To calculate the average color, we'll follow these steps:

1. Parse the text into words
2. Check every word to see if it names a color in our vector space. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs steps 1-3:

In [202]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)
# use word.lower_ to normalize case
drac_colors = [colors[word.lower_] for word in dracula if word.lower_ in colors]
avg_color = meanv(drac_colors)
print(avg_color)

[147.44839067702551, 113.65371809100999, 100.13540510543841]


Now, we'll pass the averaged color vector to the `closest()` function, yielding... well, it's just a brown mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [203]:
closest(colors, avg_color)

['reddish grey',
 'brownish grey',
 'brownish',
 'brown grey',
 'mocha',
 'grey brown',
 'puce',
 'dull brown',
 'pinkish brown',
 'dark taupe']

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*. The result definitely reflects the content of the story, so maybe we're on to something here.

In [204]:
resp = requests.get('http://www.gutenberg.org/cache/epub/1952/pg1952.txt')
yellow = nlp(resp.text)
wallpaper_colors = [colors[word.lower_] for word in yellow if word.lower_ in colors]
avg_color = meanv(wallpaper_colors)
closest(colors, avg_color)

['sickly yellow',
 'piss yellow',
 'puke yellow',
 'vomit yellow',
 'dirty yellow',
 'mustard yellow',
 'dark yellow',
 'olive yellow',
 'macaroni and cheese',
 'pea']

Exercise for the reader: Use the vector arithmetic functions to rewrite a text, making it...

* more blue (i.e., add `colors['blue']` to each occurrence of a color word); or
* more light (i.e., add `colors['white']` to each occurrence of a color word); or
* darker (i.e., attenuate each color. You might need to write a vector multiplication function to do this one right.)

## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing. They're also, for many purposes, *functionally identical*. Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [205]:
for cname in closest(colors, colors['mauve']):
    print(cname, "trousers")

mauve trousers
dusty rose trousers
dusky rose trousers
brownish pink trousers
old pink trousers
reddish grey trousers
dirty pink trousers
old rose trousers
light plum trousers
ugly pink trousers


That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about... arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words? Maybe you can see where I'm going with this. What if we made a *really big* spreadsheet that had one column for every context for every word in a given source text. Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

Such a spreadsheet might look something like this:

![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    
Because there are ten possible contexts, this is a ten dimensional space! It might be strange to think of it, but you can do vector arithmetic on vectors with ten dimensions just as easily as you can on vectors with two or three dimensions, and you could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

Of course, in a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. It's difficult enough working with a vector space of ten dimensions, let alone a vector space of a million dimensions! It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this tutorial.

## Word vectors in spaCy

Okay, let's have some fun with real word vectors. We're going to use the GloVe vectors that come with spaCy to creatively analyze and manipulate the text of Bram Stoker's *Dracula*. First, make sure you've got `spacy` imported:

In [206]:
dracula[560]

hour

In [207]:
_.vector

array([ 1.0349e-02,  1.9876e-01,  4.6758e-01,  1.1855e-01,  4.9996e-01,
       -3.1519e-01, -3.2802e-01, -1.7293e-01,  2.5819e-01,  2.2362e+00,
        6.9453e-02,  3.0208e-01, -2.3887e-01, -1.2615e-01, -1.1964e-01,
        2.7423e-01, -4.4399e-01,  1.4778e+00, -2.0461e-01, -6.0527e-01,
       -3.9626e-01,  1.8723e-01, -3.6010e-01, -3.5477e-01,  1.2472e-02,
        1.5440e-02,  1.7229e-02,  1.3381e-01,  5.4751e-01, -1.5930e-01,
        2.5280e-01,  1.4952e-01, -4.2010e-01, -4.9675e-01,  1.2056e-02,
        2.2982e-01, -1.4856e-01,  4.0397e-01, -9.7048e-03,  6.8103e-01,
        8.9441e-02,  1.4817e-01,  3.6700e-02,  1.3173e-01, -2.8417e-01,
        1.9149e-01, -3.7163e-01,  2.3378e-01,  2.1285e-01, -6.2602e-01,
        2.6401e-01, -3.7330e-01,  5.0246e-01,  2.4712e-01, -6.9188e-01,
        9.7506e-02,  2.1911e-01,  5.0629e-02,  5.0400e-01, -9.6926e-03,
       -6.5704e-02, -7.3404e-02, -1.7547e-02, -9.3765e-02,  1.7430e-01,
       -7.8283e-02,  4.0286e-01,  2.5159e-01, -3.3984e-01,  2.57

In [165]:
# previously we've used the _sm model, which doesn't include all vectors.
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz
import en_core_web_lg
nlp = en_core_web_lg.load()

In [209]:
resp = requests.get('http://www.gutenberg.org/cache/epub/345/pg345.txt')
dracula = nlp(resp.text)


In [210]:
sentence = 'Barbie loves deoxyribonucleic asdfasdfasdfasdf is really spunky'

In [211]:
words = nlp(sentence)

In [212]:
# type(words)
dna = words[5]

In [213]:
dna.pos_

'ADV'

And the cell below creates a list of unique words (or tokens) in the text, as a list of strings.

In [214]:
# all of the words in the text file
tokens = list(set([w.text for w in dracula if w.is_alpha]))

You can see the vector of any word in spaCy's vocabulary using the `vocab` attribute, like so:

In [215]:
nlp.vocab['alligator'].vector

array([-4.9304e-01,  1.2459e-01,  2.6142e-01,  3.8393e-02,  1.4018e-01,
        5.7285e-01, -5.0449e-01,  1.5094e-01,  7.3356e-02,  2.3308e-01,
       -1.6048e-01, -5.1184e-01, -1.4028e-01, -3.0110e-01, -4.3446e-01,
       -2.8257e-02,  1.9100e-03,  6.5157e-01,  1.4855e-01, -3.7255e-01,
       -4.6619e-01, -1.7223e-01, -5.1794e-01, -2.5453e-01, -2.6785e-01,
       -6.7776e-02, -3.0085e-01, -3.3212e-01, -1.5862e-01, -1.2336e-01,
       -3.8935e-01, -3.5551e-01, -3.6182e-01,  2.3197e-02, -1.7486e-01,
        2.2345e-01,  6.7557e-01,  9.6939e-03, -9.5640e-02,  1.2073e-01,
       -1.2016e-01, -1.6161e-01,  1.0661e-01, -4.5233e-01,  4.5991e-01,
        1.0367e-02, -1.0842e-01, -3.6163e-01,  2.5105e-01,  9.5780e-03,
       -6.1363e-01,  2.7494e-01,  2.1783e-01, -1.0663e-01, -9.5146e-03,
        2.7570e-01,  2.5888e-02, -5.6749e-02,  6.7667e-03,  5.3735e-01,
       -1.2224e-01,  8.8448e-02,  4.6094e-01,  4.6881e-02, -6.6557e-01,
       -3.9493e-01,  2.7362e-01,  1.2302e-01,  2.8594e-01, -2.52

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [216]:
def vec(s):
    return nlp.vocab[s].vector

### Cosine similarity and finding closest neighbors

In [217]:
from sklearn.metrics.pairwise import cosine_similarity

The following cell shows that the cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`, thereby demonstrating that the vectors are working how we expect them to:

In [218]:
(cosine_similarity(vec('dog').reshape(1, -1), vec('puppy').reshape(1, -1))[0][0] > 
    cosine_similarity(vec('trousers').reshape(1, -1), vec('octopus').reshape(1, -1))[0][0])

True

The following cell defines a function that iterates through a list of tokens and returns the token whose vector is most similar to a given vector.

In [219]:
def spacy_closest(token_list, vec_to_check, n=10):
    return sorted(token_list,
                  key=lambda x: cosine_similarity(vec_to_check.reshape(1, -1), 
                                                  vec(x).reshape(1, -1))[0][0],
                  reverse=True)[:n]

Using this function, we can get a list of synonyms, or words closest in meaning (or distribution, depending on how you look at it), to any arbitrary word in spaCy's vocabulary. In the following example, we're finding the words in *Dracula* closest to "basketball":

In [220]:
# what's the closest equivalent of basketball?
spacy_closest(tokens, vec("basketball"))

['tennis',
 'coach',
 'game',
 'teams',
 'Junior',
 'junior',
 'Team',
 'school',
 'boys',
 'leagues']

### Fun with spaCy, Dracula, and vector arithmetic

Now we can start doing vector arithmetic and finding the closest words to the resulting vectors. For example, what word is closest to the halfway point between day and night?

In [221]:
# halfway between day and night
spacy_closest(tokens, np.array(meanv([vec("day"), vec("night")])))

['night',
 'Day',
 'day',
 'evening',
 'Evening',
 'morning',
 'Morning',
 'afternoon',
 'nights',
 'Nights']

Variations of `night` and `day` are still closest, but after that we get words like `evening` and `morning`, which are indeed halfway between day and night!

Here are the closest words in _Dracula_ to "wine":

In [222]:
spacy_closest(tokens, vec("wine"))

['wine',
 'beer',
 'bottle',
 'Drink',
 'drink',
 'cellar',
 'fruit',
 'bottles',
 'brandy',
 'taste']

If you subtract "alcohol" from "wine" and find the closest words to the resulting vector, you're left with simply a lovely dinner:

In [223]:
spacy_closest(tokens, np.array(subtractv(vec("wine"), vec("alcohol"))))

['wine',
 'cellar',
 'exquisite',
 'fabulous',
 'splendid',
 'magnificent',
 'delightful',
 'Dinner',
 'dinner',
 'sparkling']

The closest words to "water":

In [224]:
spacy_closest(tokens, vec("water"))

['water',
 'waters',
 'salt',
 'Salt',
 'pond',
 'dry',
 'liquid',
 'ocean',
 'boiling',
 'heat']

But if you add "frozen" to "water," you get "ice":

In [225]:
spacy_closest(tokens, np.array(addv(vec("water"), vec("frozen"))))

['water',
 'cold',
 'ice',
 'salt',
 'Salt',
 'dry',
 'fresh',
 'liquid',
 'boiling',
 'milk']

You can even do analogies! For example, the words most similar to "grass":

In [226]:
spacy_closest(tokens, vec("grass"))

['grass',
 'lawn',
 'trees',
 'greens',
 'grassy',
 'GARDEN',
 'garden',
 'sand',
 'foliage',
 'tree']

If you take the difference of "blue" and "sky" and add it to grass, you get the analogous word ("green"):

In [227]:
# analogy: blue is to sky as X is to grass
blue_to_sky = subtractv(vec("blue"), vec("sky"))
spacy_closest(tokens, np.array(addv(blue_to_sky, vec("grass"))))

['grass',
 'GREEN',
 'Green',
 'green',
 'yellow',
 'Red',
 'red',
 'purple',
 'lawn',
 'pink']

## Further resources

* [Word2vec](https://en.wikipedia.org/wiki/Word2vec) is another procedure for producing word vectors which uses a predictive approach rather than a context-counting approach. [This paper](http://clic.cimec.unitn.it/marco/publications/acl2014/baroni-etal-countpredict-acl2014.pdf) compares and contrasts the two approaches. (Spoiler: it's kind of a wash.)
* If you want to train your own word vectors on a particular corpus, the popular Python library [gensim](https://radimrehurek.com/gensim/) has an implementation of Word2Vec that is relatively easy to use. [There's a good tutorial here.](https://rare-technologies.com/word2vec-tutorial/)
* When you're working with vector spaces with high dimensionality and millions of vectors, iterating through your entire space calculating cosine similarities can be a drag. I use [Annoy](https://pypi.python.org/pypi/annoy) to make these calculations faster, and you should consider using it too.